In [87]:
# File for implementing Naive Bayes and random forest with bag of words

In [1]:
# Libraries
import collections, re
import numpy as np
import pandas as pd
import treeUtil
import cPickle
from math import log
from nltk.corpus import stopwords

import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline

/Users/danielalpert/anaconda/lib/python2.7/site-packages/matplotlib/colors.py:680: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  not cbook.is_string_like(colors[0]):


In [2]:
from __future__ import print_function
from __future__ import division
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn
import sklearn.multiclass
import sklearn.svm
import sklearn.cross_validation
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.externals.joblib
import sklearn.metrics
import sklearn.feature_extraction.text
from nltk.tokenize import TreebankWordTokenizer
import utils
import argparse
import h5py
if utils.is_python3():
    import configparser as ConfigParser
else:
    import ConfigParser
    import scipy

/Users/danielalpert/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
dat = pd.read_excel('all_sentences.xlsx')

In [4]:
print(dat.shape)
dat.head()

(5372, 3)


,label,words,words_clean
0.0,0,Forcing middle-class workers to bear a greater...,forcing middle class workers bear greater shar...
1.0,0,Because it would not be worthwhile to bring a ...,would worthwhile bring case arbitration clause...
2.0,0,"Indeed , Lind argues that high profits and hig...",indeed lind argues high profits high wages rei...
3.0,0,"In fairness , it should be noted that he devot...",fairness noted devotes entire chapter new york...
4.0,0,Psychological tactics are social control techn...,psychological tactics social control technique...


In [38]:
# Function to clean word data--removes stopwords, makes lowercase, removes numbers
def clean_words(sentences):
    words_clean = np.full(len(sentences), None)
    for i, words in enumerate(sentences):
        word_list = re.split('\W+', words)
        words1 = [word.lower() for word in word_list if word.lower() not in stopwords.words('english')]
        words2 = [word for word in words1 if not any(char.isdigit() for char in word)]
        words_clean[i] = (' '.join(words2)).strip()
    return words_clean

### Naive Bayes

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#### First way

In [20]:
# Train test split
vectorizer = CountVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(dat['words_clean'])

X_train, X_test, y_train, y_test = train_test_split(X, dat['label'], test_size=0.25, random_state=12)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4029, 102236)
(1343, 102236)
(4029,)
(1343,)


In [21]:
mnb = MultinomialNB()
nb_model = mnb.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)
#y_pred = mnb.fit(X_train, train['label']).predict(X_test)
print("Correct labels out of %d points : %d"
      % (X_test.shape[0],(y_test == y_pred).sum()))
print('Accuracy:', (y_test == y_pred).sum()/X_test.shape[0])

Correct labels out of 1343 points : 755
Accuracy: 0.562174236783


In [22]:
probs = nb_model.predict_proba(X_test)
y_pred = []
for row in probs:
    if row[1] > 0.31:
        y_pred.append(1)
    else:
        y_pred.append(np.argmax(row))
np.mean(y_pred == y_test)

0.57110945644080413

In [23]:
def print_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("%s: %s" % (class_label,
              " ".join(feature_names[j] for j in top10)))

In [24]:
print_top10(vectorizer, nb_model, [0,1,2])

0: economic new energy health public people government care tax would
1: make good us even people jerusalem new one said would
2: state new federal make economic one free people would government


#### Second way

In [25]:
def return_acc(probs, y_test, thresh):
    y_pred = []
    for row in probs:
        if row[1] > thresh:
            y_pred.append(1)
        else:
            y_pred.append(np.argmax(row))
    #print(len(y_pred), len(y_test))
    return y_pred, np.mean(np.array(y_pred) == y_test)

In [28]:
from sklearn.pipeline import Pipeline
nb_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), 
                     ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])

#### Tune accuracy parameter

In [29]:
accuracy = [0]*50
for j in range(20):
    X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
    nb_model.fit(X_train, y_train) 
    threshes = np.arange(0.0, 0.5, 0.01)
    for i, thresh in enumerate(threshes):
        probs = nb_model.predict_proba(X_test)
        accuracy[i] += return_acc(probs, y_test, thresh)[1]

new_acc = [acc/20 for acc in accuracy]
print(new_acc)
np.argmax(new_acc)

[0.23641439205955334, 0.23672456575682382, 0.24317617866004956, 0.2616935483870968, 0.29255583126550866, 0.33455334987593061, 0.37630272952853588, 0.41861042183622832, 0.4561414392059554, 0.4850806451612904, 0.50660669975186101, 0.52651985111662536, 0.54209057071960287, 0.55158188585607937, 0.55496277915632741, 0.55614143920595538, 0.55719602977667493, 0.55514888337468993, 0.5533498759305211, 0.55046526054590572, 0.54621588089330031, 0.54311414392059554, 0.53911290322580641, 0.5349565756823822, 0.53176178660049622, 0.52844292803970216, 0.52546526054590559, 0.52264267990074453, 0.51932382133995048, 0.51656327543424307, 0.51395781637717108, 0.51225186104218368, 0.51057692307692304, 0.50896401985111672, 0.50710297766749379, 0.50511786600496278, 0.50397022332506203, 0.50341191066997515, 0.50300868486352346, 0.50266749379652609, 0.50260545905707188, 0.50254342431761789, 0.50248138957816368, 0.50248138957816368, 0.50248138957816368, 0.50248138957816368, 0.50248138957816368, 0.502481389578163

16

In [30]:
new_acc[17]

0.55514888337468993

In [32]:
plt.plot(threshes, new_acc)
plt.title('Neutral Threshold vs. Accuracy')
plt.xlabel('Neutral threshold')
plt.ylabel('Accuracy')
plt.axvline(x=0.17, c='r', linewidth=0.5, ls='--')
plt.show()

AttributeError: 'module' object has no attribute 'popall'

#### Tune ngrams

In [33]:
avg_acc = [None]*4
for i in range(len(avg_acc)):
    nb_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,i+1))), 
                     ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
    #print(nb_model_2)
    accuracy = [0]*20
    for j in range(len(accuracy)):
        X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
        nb_model.fit(X_train, y_train) 
        probs = nb_model.predict_proba(X_test)
        accuracy[j] += return_acc(probs, y_test, 0.17)[1]
    avg_acc[i] = sum(accuracy)/len(accuracy)

In [34]:
avg_acc

[0.56498138957816368,
 0.55514888337468993,
 0.55117866004962768,
 0.54900744416873448]

In [35]:
# Best model
X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3)
nb_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), 
                     ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
nb_model.fit(X_train, y_train)
probs = nb_model.predict_proba(X_test)
ret = return_acc(probs, y_test, 0.17)
print(ret[1])
print((abs(y_test - ret[0]) <= 1).sum()/X_test.shape[0])

0.532878411911
0.717741935484


In [39]:
# Test sentences
text = ['Democrats are the best party', 'Trump is a great president', 'Red sox won again', 
        'I eat at burger king twice a week', 'This bill has the potential to hurt housing for low-income minorities',
        'By carrying this out, the american people are attacking freedom', 'black lives matter', 'blue lives matter',
        'white lives matter', 'trickle down economics saved our economy', 'regulations saved our economy',
        'illegal immigrants actually help our economy more than hurt it', 'i am neutral 6']
labels = pd.Series([0,2,1,1,0,2,0,2,2,2,0,0,1])
clean_text = clean_words(text)

In [40]:
probs = nb_model.predict_proba(clean_text)
#print(type(y_pred), type(labels.tolist()))
return_acc(probs, labels, 0.17)

([0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1], 0.69230769230769229)

In [41]:
diffs = [row[0]-row[2] for row in probs]
diffs

[0.053439099296746284,
 0.25453105286842331,
 0.028709354909863261,
 0.14745926939266624,
 0.39219901660444884,
 -0.031046310260656573,
 0.2695985764193724,
 0.14205940778550102,
 0.21644608982092911,
 0.21865346741242619,
 0.037614802880942377,
 0.24418034849431458,
 0.17753877046382754]

#### Third way

In [43]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [49]:
X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.2, random_state=0)
nb_model_3 = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), max_features=1000)), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
nb_model_3.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        st...False,
         use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [50]:
#clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(nb_model_3, dat['words_clean'], dat['label'], cv=10)
np.mean(scores)

0.53597278477500976

In [51]:
text = ['Democrats are the best party', 'Trump is a great president', 'Red sox won again', 
        'I eat at burger king twice a week', 'This bill has the potential to hurt housing for low-income minorities',
        'By carrying this out, the american people are attacking freedom', 'black lives matter', 'blue lives matter',
        'white lives matter', 'trickle down economics saved our economy', 'regulations saved our economy',
        'illegal immigrants actually help our economy more than hurt it', 'i am neutral',
        'invertebrates have the highest acidity in the animal kingdom', 'hello']
new_text = []
for string in text:
    print(string)
    words = (string.lower()).split(' ')
    new_text.append(' '.join([word for word in words if word not in stopwords.words('english')]))

new_text

Democrats are the best party
Trump is a great president
Red sox won again
I eat at burger king twice a week
This bill has the potential to hurt housing for low-income minorities
By carrying this out, the american people are attacking freedom
black lives matter
blue lives matter
white lives matter
trickle down economics saved our economy
regulations saved our economy
illegal immigrants actually help our economy more than hurt it
i am neutral
invertebrates have the highest acidity in the animal kingdom
hello


['democrats best party',
 'trump great president',
 'red sox',
 'eat burger king twice week',
 'bill potential hurt housing low-income minorities',
 'carrying out, american people attacking freedom',
 'black lives matter',
 'blue lives matter',
 'white lives matter',
 'trickle economics saved economy',
 'regulations saved economy',
 'illegal immigrants actually help economy hurt',
 'neutral',
 'invertebrates highest acidity animal kingdom',
 'hello']

In [52]:
probs = nb_model_3.predict_proba(new_text)

### Random Forest

In [53]:
from sklearn import tree
from sklearn import ensemble

In [54]:
X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=19)

In [55]:
tree_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1), max_features=500)), 
                     ('tfidf', TfidfTransformer()), ('clf', tree.DecisionTreeClassifier(max_depth=500)),])
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
probs = tree_model.predict_proba(X_test)

In [56]:
return_acc(probs, y_test, .34)[1]

0.49007444168734493

In [57]:
accuracy = [0]*50
for j in range(20):
    X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
    tree_model.fit(X_train, y_train) 
    threshes = np.arange(0.0, 0.5, 0.01)
    for i, thresh in enumerate(threshes):
        probs = tree_model.predict_proba(X_test)
        accuracy[i] += return_acc(probs, y_test, thresh)[1]

new_acc = [acc/20 for acc in accuracy]
print(new_acc)
np.argmax(new_acc)

[0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48901985111662538, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.48933002481389576, 0.4893300248

34

#### Tune 

In [58]:
num_feats = [10,30,50,100,200,300,500,800,1000,1500]
avg_acc = [None]*len(num_feats)
for i, feats in enumerate(num_feats):
    tree_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1), max_features=feats)), 
                     ('tfidf', TfidfTransformer()), ('clf', tree.DecisionTreeClassifier(max_depth=500)),])
    #print(nb_model_2)
    accuracy = [0]*20
    for j in range(len(accuracy)):
        X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
        tree_model.fit(X_train, y_train) 
        probs = tree_model.predict_proba(X_test)
        accuracy[j] += return_acc(probs, y_test, 0.34)[1]
    avg_acc[i] = sum(accuracy)/len(accuracy)

In [59]:
avg_acc

[0.42788461538461531,
 0.42673697270471467,
 0.44630893300248137,
 0.45908808933002482,
 0.47354218362282874,
 0.48067617866004958,
 0.48725186104218371,
 0.49230769230769234,
 0.49441687344913143,
 0.48874069478908194]

### Random Forest

In [60]:
X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=19)

In [61]:
rf_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1), max_features=1500)), 
                     ('tfidf', TfidfTransformer()), ('clf', ensemble.RandomForestClassifier(max_depth=200, n_estimators=50)),])
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
np.mean(y_pred == y_test)

0.56079404466501237

In [62]:
accuracy = [0]*50
for j in range(20):
    X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
    rf_model.fit(X_train, y_train) 
    threshes = np.arange(0.0, 0.5, 0.01)
    for i, thresh in enumerate(threshes):
        probs = rf_model.predict_proba(X_test)
        acc = return_acc(probs, y_test, thresh)[1]
        print(acc)
        accuracy[i] += acc

new_acc = [acc/20 for acc in accuracy]
print(new_acc)
np.argmax(new_acc)

0.25682382134
0.25682382134
0.285359801489
0.287841191067
0.323200992556
0.326302729529
0.357320099256
0.364143920596
0.383995037221
0.390818858561
0.41253101737
0.424937965261
0.444168734491
0.458436724566
0.468362282878
0.483250620347
0.486972704715
0.494416873449
0.501240694789
0.50682382134
0.514267990074
0.519851116625
0.526054590571
0.532878411911
0.5341191067
0.536600496278
0.543424317618
0.549007444169
0.549627791563
0.55459057072
0.557692307692
0.56017369727
0.56141439206
0.565136476427
0.564516129032
0.566377171216
0.564516129032
0.566377171216
0.565136476427
0.565136476427
0.566377171216
0.566377171216
0.565756823821
0.564516129032
0.563275434243
0.563275434243
0.563275434243
0.563275434243
0.563275434243
0.563275434243
0.239454094293
0.239454094293
0.274193548387
0.276054590571
0.305210918114
0.31141439206
0.341191066998
0.349875930521
0.382133995037
0.393920595533
0.414392059553
0.422456575682
0.444168734491
0.451612903226
0.468362282878
0.478908188586
0.488833746898
0.498

45

In [63]:
new_acc[41]

0.56299627791563267

In [64]:
num_feats = [10,30,50,100,200,300,500,800,1000,1500,2000,2500,3000,3500,4000]
avg_acc = [None]*len(num_feats)
for i, feats in enumerate(num_feats):
    rf_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))), 
        ('tfidf', TfidfTransformer()), ('clf', ensemble.RandomForestClassifier(max_depth=500, max_features=feats, n_estimators=10)),])
    #print(nb_model_2)
    accuracy = [0]*20
    for j in range(len(accuracy)):
        X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
        rf_model.fit(X_train, y_train) 
        probs = rf_model.predict_proba(X_test)
        accuracy[j] += return_acc(probs, y_test, 0.28)[1]
    avg_acc[i] = sum(accuracy)/len(accuracy)

In [67]:
num_feats[np.argmax(avg_acc)]

200

In [68]:
depths = [1,2,3,5,10,30,50,70,100,200,300,500,800,1000,1500]
avg_acc = [None]*len(depths)
for i, depth in enumerate(depths):
    rf_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))), 
        ('tfidf', TfidfTransformer()), ('clf', ensemble.RandomForestClassifier(max_depth=depth, max_features=1500, n_estimators=10)),])
    #print(nb_model_2)
    accuracy = [0]*20
    for j in range(len(accuracy)):
        X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
        rf_model.fit(X_train, y_train) 
        probs = rf_model.predict_proba(X_test)
        accuracy[j] += return_acc(probs, y_test, 0.28)[1]
    avg_acc[i] = sum(accuracy)/len(accuracy)

In [69]:
depths[np.argmax(avg_acc)]

500

In [70]:
num_estimators = [1,2,3,5,10,20,30]
avg_acc = [None]*len(num_estimators)
for i, n in enumerate(num_estimators):
    rf_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))), 
        ('tfidf', TfidfTransformer()), ('clf', ensemble.RandomForestClassifier(max_depth=200, max_features=1500, n_estimators=n)),])
    #print(nb_model_2)
    accuracy = [0]*20
    for j in range(len(accuracy)):
        X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
        rf_model.fit(X_train, y_train) 
        probs = rf_model.predict_proba(X_test)
        accuracy[j] += return_acc(probs, y_test, 0.28)[1]
    avg_acc[i] = sum(accuracy)/len(accuracy)

In [71]:
num_estimators[np.argmax(avg_acc)]
avg_acc

[0.48979528535980144,
 0.48874069478908178,
 0.47732630272952853,
 0.51913771712158796,
 0.52679900744416874,
 0.5367866004962778,
 0.5436104218362281]

In [72]:
rf_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))), 
        ('tfidf', TfidfTransformer()), ('clf', ensemble.RandomForestClassifier(max_depth=200, max_features=1500, n_estimators=50)),])
accuracy = [0]*20
for j in range(len(accuracy)):
    X_train, X_test, y_train, y_test = train_test_split(dat['words_clean'], dat['label'], test_size=0.3, random_state=j)
    rf_model.fit(X_train, y_train) 
    probs = rf_model.predict_proba(X_test)
    accuracy[j] += return_acc(probs, y_test, 0.28)[1]
avg_acc = sum(accuracy)/len(accuracy)

In [73]:
avg_acc

0.54655707196029779

In [74]:
rf_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))), 
        ('tfidf', TfidfTransformer()), ('clf', ensemble.RandomForestClassifier(max_depth=200, max_features=1500, n_estimators=50)),])